COMP 215 - LAB 2 Records (NEO)
----------------
#### Name: Stuart Kwan
#### Date: 2024-01-16

This lab exercise is mostly a review of strings, tuples, lists, dictionaries, and functions.

**Building on new concepts from lab 1**:
  * `datetime.date` objects represent a calendar date
  * *list comprehension* provides a compact way to represent map and filter algorithms

**New Python Concepts**:
  * *f-string* simplifies string formatting operations

As usual, the first code cell simply imports all the modules we'll be using...

In [ ]:
import datetime, json, requests
from pprint import pprint    # Pretty Print - built-in python function to nicely format data structures

We'll answer some questions about [Near Earth Objects](https://cneos.jpl.nasa.gov/)
> using NASA's API:  [https://api.nasa.gov/](https://api.nasa.gov/#NeoWS)

You should register for your own API key, (but may use the DEMO_KEY to get started).

First we need a short tutorial on python dates and [f-strings](https://realpython.com/python-f-strings/)...

In [ ]:
today = datetime.date.today()   # get a date object representing today's date
print(today, type(today))
formatted_date = f'Today is: {today}'   # A format string - notice how the variable `today` is formatted into the string
print(formatted_date)

### Make a query

Let's get some data from the NEO database...
Here's a query that gets the observation "feed" for today.
(Note: I hard-coded the date below to lock down the data for the lab - ideally used `today()` so the notebook is always up-to-date.)

In [ ]:
API_KEY = 'DEMO_KEY'  # substitute your API key here

today = '2023-01-09'  #  Future enhancement:  str(datetime.date.today())   # Today's date as a string!
# Use an f-string here to "format" the date and API key varaibles.
url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={today}&end_date={today}&api_key={API_KEY}'

response = requests.request("GET", url, headers={}, data={})

data = json.loads(response.text)  # recall json.loads for lab 1

# TIP: print(data) to see the whole data structure returned, here we grab just the list of NEO's:
n_results = data['element_count']
neos = data['near_earth_objects'][today]
print(f'{n_results} Near Earth Objects found for {today}')
pprint(neos[:5])

Next we extract just the potentially hazerdous asteroids, using a Comp115-style list accumulator *loop*:

In [ ]:
hazards =  []
for item in neos:
  if item['is_potentially_hazardous_asteroid'] is True:
    hazards.append(item)
print(f'{len(hazards)} potentially hazardous asteroids identified.')

## Exercise 1

In the code cell below, **re-write the accumulator loop above** as a [list comprehension](https://realpython.com/lessons/list-comprehensions-overview/) that implements a ["filter"](https://youtu.be/hUes6y2b--0)
Notice how this provides a concise way to "filter" items of interest from a larger data set.

In [ ]:
# Ex. 1 your code here

num_hazards = [item['is_potentially_hazardous_asteroid'] for item in neos if item['is_potentially_hazardous_asteroid'] is True]
print(f'{len(num_hazards)} potentially hazardous asteroids identified.')

3 potentially hazardous asteroids identified.


## Fetch Complete Data for One Asteroid

Notice that the record for each `neo` is a dictionary with `id` field that uniquely identifies this record in the database.

We can use this `id` to fetch complete orbital and close approach data for the NEO.

For example, this query fetches the complete data set for the first hazardous asteroid...


In [ ]:
id = hazards[0]['id']
url = f'https://api.nasa.gov/neo/rest/v1/neo/{id}?api_key={API_KEY}'
response = requests.request("GET", url, headers={}, data={})
data = json.loads(response.text)

pprint(data)

Notice that the `miss_distance` field contains the distance (in various units) by which the NEO missed an "orbiting body".

## Exercise 2

In the code cell below, write a python function that takes a list of "close approach data" as a parameter,
and returns a 2-tuple with the (date, miss km) of the closest approach to Earth in the list (where "miss km" is the miss distance in km).

Hints:
* notice the input is a list of dictionaries.  Each dictionary has a 'close_approach_date", "orbiting_body", and 'miss_distance' field.
* we are only interested in the closest approach to "Earth"
* use a loop if that is easier to understand - we will look at more compact algorithms to solve this problem in class.

Add at least one unit test to check your work - note the test data only needs dictionaries with the fields your function actually uses.


In [ ]:
# Ex. 2 your code here

approach_data = data['close_approach_data']

def closest_miss_distance(approach_list):
  date = ''
  dist = float(approach_list[0]['miss_distance']['astronomical']) # set distance to the first miss distance
  for i in approach_list: # loop through approach_list
    prev_dist = float(i['miss_distance']['astronomical'])
    if prev_dist < dist:
      dist = prev_dist
      date = i['close_approach_date']
  return dist, date

closest_miss_distance(approach_data)

(0.0287693216, '1972-06-25')

## Challenge - Take your skills to the next level...
## Exercise 3

In the code cell below, write a complete program that:
 1. fetches the list of NEO's for this week.
 2. for each NEO, fetch it's complete orbital data and determine its closest approach to Earth
 3. identify which NEO from this week's data makes the closet approach to earth
 4. print a nice message with information about the NEO, which it will approach the Earth, and how close it will come.

Hints:
* you'll need the start and end date - end date is today, see if you can use a [`timedelta`](https://docs.python.org/3/library/datetime.html#timedelta-objects) object to ge the start date (you can do basic "date math" with `timedelta` and `date` objects!)
* you may need to modify the function we wrote in Ex. 2 to return a triple with the NEO's id included;
* lots of opportunity here for more practice with list comprehensions


In [32]:
from typing import ItemsView
# Ex. 3 (challenge) your code here
import datetime, json, requests
from pprint import pprint


API_KEY = 'vN3UA35SEZcPWPRTlWcC3FY2fbSpHXrjdEpwoeNA'
day = datetime.timedelta(days=1)
seven_days = datetime.timedelta(days=7)
today = datetime.date.today()
current_week = today - seven_days


def get_neos():
  '''Gets the near earth objects of the last week'''
  start = str(current_week)
  end = str(today)
  url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start}&end_date={end}&api_key={API_KEY}'
  response = requests.request("GET", url, headers={}, data={})
  data = json.loads(response.text)
  return data


def single_neo_info(date, data):
  '''Returns the distance, date, and id of the closest neo to earth'''
  date_str = str(date)
  neo_list = data['near_earth_objects'][date_str]
  neo_dist = float(neo_list[0]['close_approach_data'][0]['miss_distance']['astronomical'])
  neo_name = ''
  neo_date = ''
  neo_id = ''

  for neo in range(len(neo_list)):
    prev_dist = float(neo_list[neo]['close_approach_data'][0]['miss_distance']['astronomical'])
    if prev_dist < neo_dist:
      neo_dist = prev_dist
      neo_name = neo_list[neo]['name']
      neo_date = neo_list[neo]['close_approach_data'][0]['close_approach_date']
      neo_id = neo_list[neo]['id']
    else:
      pass
  return neo_dist, neo_name, neo_date, neo_id


def closest_weekly_neo():
  '''Loops throught the last week of neos and returns the neo closest to Earth'''
  neo_data = get_neos()
  date = current_week
  current_neo = single_neo_info(str(date), neo_data)

  for i in range(7):
    prev_neo = single_neo_info(str(date), neo_data)
    date = date + day
    if prev_neo[0] < current_neo[0]:
      current_neo = prev_neo
  return current_neo



neo = closest_weekly_neo()
print('The closest near Earth object', neo[1], 'with the ID', neo[3], ', flew by on', neo[2], 'at approximately', neo[0], 'astronomical units away from Earth.')


The closest near Earth object (2024 BC) with the id 54421065 flew by on 2024-01-20 at approximately 0.0056631064 astronomical units.
